# Equilibrium, P = 2.3, T = 5.2

## Mesogens with Nanoparticles

In [1]:
# -----Import essential packages

from __future__ import division
import hoomd
import hoomd.md

In [2]:
#-----Define important variables

temp = 5.2;
pressure = 2.3;
steps = 1e5;

In [3]:
#-----Declare the file from wich we'll extract the configuration of the system.

ramp_file = "T_CM&NP_" + str(temp) + "_P_" + str(pressure) + "_ramp.gsd"

In [4]:
#-----Start from a gsd file

hoomd.context.initialize("--mode=cpu", memory_traceback = True);

#-----Select the trayectory of the center of mass of particles

system = hoomd.init.read_gsd(ramp_file,frame = -1)

#-----Define each rigid body type in the local coordinate system of the body.

rigid = hoomd.md.constrain.rigid();
rigid.set_param('M', 
                types=['A']*8,
                positions=[(-4,0,0),(-3,0,0),(-2,0,0),(-1,0,0),
                           (1,0,0),(2,0,0),(3,0,0),(4,0,0)]);

#-----Instruct the rigid constraint to create the constituent particles

rigid.create_bodies();

#-----Define the potential energy

nl = hoomd.md.nlist.tree()

lj = hoomd.md.pair.lj(r_cut = 3.5, nlist=nl)
lj.set_params(mode='shift')

#-----Define interaction with species in the binary mixture

lj.pair_coeff.set('NP','NP', epsilon = 1.0, sigma = 5.0);
lj.pair_coeff.set('M' ,'M' , epsilon = 1.0, sigma = 1.0);
lj.pair_coeff.set('A' ,'A' , epsilon = 1.0, sigma = 1.0);
lj.pair_coeff.set('M' ,'A' , epsilon = 1.0, sigma = 1.0);
lj.pair_coeff.set('NP','M' , epsilon = 1.0, sigma = 3.0);
lj.pair_coeff.set('NP','A' , epsilon = 1.0, sigma = 3.0);

#-----Select an standard integrator.

hoomd.md.integrate.mode_standard(dt = 0.005);

#------Define some groups and make their union.

nanoparticles = hoomd.group.type(name = 'Nano_Particles', type = 'NP');
mesogens = hoomd.group.rigid_center();
groupNP_mes = hoomd.group.union(name = "NP_Mes", a = nanoparticles, b = mesogens);

#----Integrate using NPT

npt = hoomd.md.integrate.npt(group = mesogens, kT = temp, tau = 10.0, tauP = 10.0, P = pressure);

#-----Write output and Run the Simulation

log_file = "T_" + str(temp) + "_P_" + str(pressure) + "_equilibrium.log"
gsd_file = "T_" + str(temp) + "_P_" + str(pressure) + "_equilibrium.gsd" 
meso_gsd_file = "T_CM_" + str(temp) + "_P_" + str(pressure) + "_equilibrium.gsd"
log = hoomd.analyze.log(filename = log_file,
                         quantities = ['num_particles',
                                     'ndof',
                                     'translational_ndof',
                                     'rotational_ndof',
                                     'potential_energy',
                                     'kinetic_energy',
                                     'translational_kinetic_energy',
                                     'rotational_kinetic_energy',
                                     'temperature',
                                     'pressure',
                                      'volume'],
                         period = 1e3,
                         overwrite = True);
gsd = hoomd.dump.gsd(gsd_file,
               period = 1e3,
               group = hoomd.group.all(),
               overwrite=True); 
meso_gsd = hoomd.dump.gsd(meso_gsd_file,
               period = 1e3,
               group = mesogens,
               overwrite=True); 

HOOMD-blue 2.6.0 DOUBLE HPMC_MIXED TBB SSE SSE2 SSE3 
Compiled: 05/29/2019
Copyright (c) 2009-2019 The Regents of the University of Michigan.
-----
You are using HOOMD-blue. Please cite the following:
* J A Anderson, C D Lorenz, and A Travesset. "General purpose molecular dynamics
  simulations fully implemented on graphics processing units", Journal of
  Computational Physics 227 (2008) 5342--5359
* J Glaser, T D Nguyen, J A Anderson, P Liu, F Spiga, J A Millan, D C Morse, and
  S C Glotzer. "Strong scaling of general-purpose molecular dynamics simulations
  on GPUs", Computer Physics Communications 192 (2015) 97--107
-----
HOOMD-blue is running on the CPU
notice(2): Group "all" created containing 1004 particles
notice(2): constrain.rigid(): Creating 1000 rigid bodies (adding 8000 particles)
-----
You are using tree neighbor lists. Please cite the following:
* M P Howard, J A Anderson, A Nikoubashman, S C Glotzer, and A Z
  Panagiotopoulos. "Efficient neighbor list calculation for mol

In [5]:
#-----Run the simulation.

hoomd.run(steps)

notice(2): -- Neighborlist exclusion statistics -- :
notice(2): Particles with 0 exclusions             : 9004
notice(2): Neighbors included by diameter          : no
notice(2): Neighbors excluded when in the same body: yes
** starting run **
Time 00:00:10 | Step 608131 / 708000 | TPS 12.9802 | ETA 02:08:13
Time 00:00:20 | Step 608284 / 708000 | TPS 15.2016 | ETA 01:49:19
Time 00:00:30 | Step 608452 / 708000 | TPS 16.7751 | ETA 01:38:54
Time 00:00:40 | Step 608611 / 708000 | TPS 15.8689 | ETA 01:44:23
Time 00:00:50 | Step 608772 / 708000 | TPS 16.0921 | ETA 01:42:46
Time 00:01:00 | Step 608937 / 708000 | TPS 16.3129 | ETA 01:41:12
Time 00:01:10 | Step 609102 / 708000 | TPS 16.4819 | ETA 01:40:00
Time 00:01:20 | Step 609269 / 708000 | TPS 16.6675 | ETA 01:38:43
Time 00:01:30 | Step 609437 / 708000 | TPS 16.6406 | ETA 01:38:43
Time 00:01:40 | Step 609607 / 708000 | TPS 16.8688 | ETA 01:37:12
Time 00:01:50 | Step 609773 / 708000 | TPS 16.4371 | ETA 01:39:35
Time 00:02:00 | Step 609940 / 7

In [6]:
#-----Get density.

system.get_metadata()

OrderedDict([('box', <hoomd.data.boxdim at 0x110b5a9b0>),
             ('particles', <hoomd.data.particle_data at 0x110c585c0>),
             ('number_density', 0.8415924534746826),
             ('bonds', <hoomd.data.bond_data at 0x110c58630>),
             ('angles', <hoomd.data.angle_data at 0x110c586d8>),
             ('dihedrals', <hoomd.data.dihedral_data at 0x110c58748>),
             ('impropers', <hoomd.data.dihedral_data at 0x110c587b8>),
             ('constraints', <hoomd.data.constraint_data at 0x110c58828>),
             ('pairs', <hoomd.data.bond_data at 0x110c58898>),
             ('timestep', 708000)])

In [7]:
#-----Get volume.

system.box.get_volume()

10698.765136052714